In [212]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import ascii
import numpy as np


In [213]:
# PRELIMINARY

# 1 - load data

data = ascii.read("tableb2.pos")  
radec=data.as_array()

# 2 - create coord list

ra=data['col1']
dec=data['col2']
start=int(20000)
stop=int(start+50)
coord_list = SkyCoord(ra=ra[start:stop], dec=dec[start:stop], unit=u.deg)


In [84]:
# METHOD 1

result = Vizier.query_region(coord_list, radius=0.2 * u.arcsec, catalog='II/246')

# Im interested only in the IDs, print them:

#result[0]['_2MASS']

# Not bad at all but this will time out for large lists such as the >70000 records we have here
# => can be done in chunks of ~1000 for instance (see Method 3)
# One drawback of this method is all the default columns of the catalog are retrieved
# whereas we are only interested in "_2MASS', so we're wasting bandwidth
# => Can we improve things (the size of the chunk manageable for isntance)
# By requesting only the "_2MASS" column is sent back?


In [81]:
# METHOD 2

# The columns to be reteieved are specified in the VizieR instanciation

v=Vizier(columns=["2MASS"], catalog="II/246")
result = v.query_region(coord_list, radius=0.2 * u.arcsec)

# But sadly this bugs


ReadTimeout: HTTPConnectionPool(host='vizier.u-strasbg.fr', port=80): Read timed out. (read timeout=60)

In [116]:
# METHOD 3: is method 1 by chunks

#lets do a smaller sample to debug
#ra=ra[:1245]
#dec=dec[:1245]

In [214]:

nra=len(ra)
ids=np.empty(nra,dtype='object_')
nchunk=1000
start=0
nsteps=int(nra/nchunk)+1

for i in np.arange(0,nsteps):
    print("step",i)
    if(start+nchunk>nra):
        nchunk=int(nra-start)
    print("chunksize",nchunk)
    
    rachunk=ra[start:start+nchunk]
    dechunk=dec[start:start+nchunk]
    coord_list = SkyCoord(ra=rachunk, dec=dechunk, unit=u.deg)
    result = Vizier.query_region(coord_list, radius=0.2 * u.arcsec, catalog='II/246')
    ids[start:start+nchunk]=(result[0]["_2MASS"].data.filled().data.astype('str'))
    start=start+nchunk

# save the result
f=open('2MASS_bychunk.csv','w')
np.savetxt(f,ids,fmt="%s")
f.close()


step 0
chunksize 1000
step 1
chunksize 1000
step 2
chunksize 1000
step 3
chunksize 1000
step 4
chunksize 1000
step 5
chunksize 1000
step 6
chunksize 1000
step 7
chunksize 1000
step 8
chunksize 1000
step 9
chunksize 1000
step 10
chunksize 1000
step 11
chunksize 1000
step 12
chunksize 1000
step 13
chunksize 1000
step 14
chunksize 1000
step 15
chunksize 1000
step 16
chunksize 1000
step 17
chunksize 1000
step 18
chunksize 1000
step 19
chunksize 1000
step 20
chunksize 1000
step 21
chunksize 1000
step 22
chunksize 1000
step 23
chunksize 1000
step 24
chunksize 1000
step 25
chunksize 1000
step 26
chunksize 1000
step 27
chunksize 1000
step 28
chunksize 1000
step 29
chunksize 1000
step 30
chunksize 1000
step 31
chunksize 1000
step 32
chunksize 1000
step 33
chunksize 1000
step 34
chunksize 1000
step 35
chunksize 1000
step 36
chunksize 1000
step 37
chunksize 1000
step 38
chunksize 1000
step 39
chunksize 1000
step 40
chunksize 1000
step 41
chunksize 1000
step 42
chunksize 1000
step 43
chunksize 100

In [ ]:
# The original example from Katharina
ra = [120.689, 150.3684, 4.9516]
dec = [-23.4578, -65.794236, 35.5631]
coord_list = SkyCoord(ra=ra, dec=dec, unit=u.deg)
catalog_list = Vizier.find_catalogs('2MASS PSC')
print({k:v.description for k,v in catalog_list.items()})
result = Vizier.query_region(coord_list, radius=0.2 * u.deg, catalog='II/246')
print(result[0])